# Lab | APIS

In this lab, you will collect historical data about the Nobel Prize winners using [this free and non-authenticated API](https://www.nobelprize.org/organization/developer-zone-2/). According to the documentation available [here](https://app.swaggerhub.com/apis/NobelMedia/NobelMasterData/2.1#/default/get_nobelPrizes). The base url is: "http://api.nobelprize.org/2.1/" followed by a string to specify what kind of information do you want to retrieve. The acceptable options are:

* nobelPrices
* nobelPrice/category/year
* laureates
* laureate/laureateID

# Getting the information using requests

Use the Python `requests`, and `json` libraries to obtain the information of ALL the Nobel Prizes. Make sure to verify that you get the proper status code (200).

The json outputs are simple plain text that need to be converted into the corresponding nested dictionary. Use the `.json()` method to cast the output into a Python dictionary.

Use the Pandas library to collect all the information into a Panda's DataFrame.

In [ ]:
import requests
import json
import pandas as pd

url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
        print("All good!")
        print("==============")
        print("\n")
        data = response.json()["nobelPrizes"]
        data = pd.DataFrame(data)

# Processing the output

Process the Pandas DataFrame in order to have only the following columns:

- category
- dateAwarded (as DateTime in "yyyy-mm-dd" format)
- prizeAmount
- prizeAmountAdjusted
- Number_of_laureates
- motivation
- laureate_ids (as a list)

In [ ]:
url = "http://api.nobelprize.org/2.1/nobelPrizes?limit=100000"

response = requests.get(url)

if response.status_code == 200:
    prizes_list = response.json()['nobelPrizes']
    data = pd.DataFrame(prizes_list)
    nobel = pd.DataFrame()
    nobel['category'] = data['category'].str['en']
    nobel['dateAwarded'] = data['dateAwarded']
    nobel['prizeAmount'] = data['prizeAmount']
    nobel['prizeAmountAdjusted'] = data['prizeAmountAdjusted']
    nobel['Number_of_laureates'] = data['laureates'].apply(lambda x: len(x) if isinstance(x, list) else 0)
    nobel['motivation'] = data['laureates'].apply(lambda x: x[0]['motivation']['en'] if isinstance(x, list) else None)
    nobel['laureate_ids'] = data['laureates'].apply(lambda x: [laureate['id'] for laureate in x] if isinstance(x, list) else [])
    
nobel

# Getting a Pandas DataFrame with the details of awarded authors/institutions

If you dive deeper and use the API to retrieve the details of some laureate_ids, you will notice that not allways the Nobel Prize was awarded to individuals. In some cases, the awards were given to institutions.

Get the unique ids from the previous datasets and prepare the following functions:

- get_name(laureate) ( it should return the english name 'fullName' of the individual or 'orgName' of the institution )

- get_gender(laureate) ( it should return the gender or 'Unknown' for individuals, and 'None' for institutions )

- get_birthdate(laureate) ( it should return the birthdate when it's avaialble or 'Unknown' otherwise )

- get_age(laureate) ( it should return the age of the awarded individual or 'Unknown' when it's not avaialble or for institutions )

- get_city(laureate) ( it should return the english name of the city when it's available or 'Unknown' otherwise )

- get_country(laureate) ( it should return the english name of the country when it's available or 'Unknown' otherwise )

- get_continent(laureate) ( it should return the english name of the continent when it's available or 'Unknown' otherwise )

- get_latitude(laureate) ( it should return the city's latitude when it's available or 'Unknown' otherwise )

- get_longitude(laureate) ( it should return the city's longitude
 when it's available or 'Unknown' otherwise )

Create the following dictionaries:

```python
laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}                        

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}
```

For each unique `laureate_id` of the previous DataFrame make an API call to get the details of the awarded individual/intitution and iterate of the previous dictionaries keys in order to add the corresponding information of each `laureate_id` in the empty lists of `laureates_dict`.

Finally, create a Pandas DataFrame named `laureates_df` using the `laureates_dict`.

In [ ]:
import time
from tqdm import tqdm


ids = [int(item) for l in df['laureate_ids'].values for item in l]
unique_ids = set(ids)

def get_name(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here


def get_gender(laureate):

    # Person
    # Your code here

    # Organizrtion
    # Your code here

def get_birthdate(laureate):

    birth_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    return birth_date

def get_age(laureate):

    birth_date = "Unknown"
    award_date = "Unknown"

    # Person
    # Your code here

    # Organization
    # Yuor code here

    # Award date
    # Your code here

def get_city(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_country(laureate):

    # Person
    # Your code here

    # Origanization
    # Your code here


def get_continent(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_latitude(laureate):

    # Person
    # Your code here

    # Organization
    # Your code here

def get_longitude(laureate):

    # Person
    # Your code here

    # Organization
    # Yuor code here


laureates_dict = {"ID": [], "Name": [], "Gender": [], \
                  "Birth_date": [], "Age": [], \
                  "City": [], "Country": [], "Continent": [], \
                  "Latitude": [], "Longitude": []}

functions_dict = {"ID": None, "Name": get_name, "Gender": get_gender, \
                  "Birth_date": get_birthdate, "Age": get_age, \
                  "City": get_city, "Country": get_country, "Continent": get_continent, \
                  "Latitude": get_latitude, "Longitude": get_longitude}

for index, id in enumerate(tqdm(unique_ids)):

    url = "https://api.nobelprize.org/2/laureate/" + str(id)
    response = requests.get(url)

    if response.status_code == 200:

        laureate = response.json()

        # Your code here

laureates_df = pd.DataFrame(laureates_dict)

laureates_df



# Country ranking

Get a ranking countries by the number of times that they had been awarded in any category.

In [ ]:
# Your code here